In [ ]:
import sys
sys.path.append("../")
print(sys.path)

In [ ]:
import yaml
import json
from json import dumps, loads
from kafka import KafkaProducer, KafkaConsumer
from fedrec.data_models.job_submit_model import JobSubmitMessage
from fedrec.utilities import registry
import fedrec
import experiments
import fl_strategies
import datasets
with open("../configs/regression.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

def init_kafka(config):
    producer_url = "{}:{}".format(
        config["producer_url"], config["producer_port"])
    return KafkaProducer(
        bootstrap_servers=[producer_url])

# serializer = registry.construct("serializer", "json")
# config = config["multiprocessing"]["communications"]

producer = init_kafka(config["multiprocessing"]["communication_interface"])
producer.send('job-request-aggregator', value=str({"job_type":"train", "results": {},"error": [1,2]}).encode('utf-8'))


In [ ]:
def init_kafka(config):
    consumer_url = "{}:{}".format(
        config["consumer_url"], config["consumer_port"])
    return KafkaConsumer(
        bootstrap_servers=[consumer_url])



In [ ]:
with open("../configs/dlrm_fl.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

ag_config = {
        # Seed for RNG used in shuffling the training data.
    "data_seed" : 100,
    # Seed for RNG used in initializing the model.
    "init_seed" : 100,
    # Seed for RNG used in computing the model's training loss.
    # Only relevant with internal randomness in the model, e.g. with dropout.
    "model_seed" : 100
}


In [2]:
from torch.optim import Optimizer
dir(Optimizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_hook_for_profile',
 'add_param_group',
 'load_state_dict',
 'state_dict',
 'step',
 'zero_grad']